<a href="https://colab.research.google.com/github/rafavidal1709/mack_ai/blob/main/Anonimizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

class Anonimizacao:
  def __init__(self, model, tokenizer=None, replace={}):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.replace = replace
    self.model_path = model
    self.tokenizer_path = tokenizer if tokenizer else model
    self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_path)
    self.model = AutoModelForTokenClassification.from_pretrained(self.model_path).to(self.device)
    self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer, device=self.device)

  def aplicar_modelo(self, texto):
    return self.nlp(texto)

  def anonimizar_texto(self, texto, resultado):
    limpo = ""
    ind = 0

    last_index = -2
    for i in resultado:
      if i['index']==last_index+1: #and i['entity'][2:]==last_entity[2:]:
        ind = i['end']
        last_index = i['index']
        last_entity = i['entity']
        continue

      if i["entity"][2:] in self.replace:
        s = self.replace[i["entity"][2:]]
      else:
        s = i["entity"][2:]
      limpo += texto[ind:i['start']]+s
      ind = i['end']

      last_index = i['index']
      last_entity = i['entity']


    limpo += texto[ind:]
    return limpo

  def anonimizar(self, texto):
    resultado = self.aplicar_modelo(texto)
    return self.anonimizar_texto(texto, resultado)

  def testar_com(self,texto):
    resultado = self.aplicar_modelo(texto)
    for i in resultado:
      print(i)
    print("\n",self.anonimizar_texto(texto, resultado))

  def testar(self, tokenizer=None, replace={}):
    if type(self)==str:
      model = Anonimizacao(self, tokenizer=tokenizer, replace=replace)
      print("\n\n")
      model.testar()
      return model

    conjunto = [
        "Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota.",
        "Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo"
    ]

    for texto in conjunto:
      print(texto,"\n")
      self.testar_com(texto)
      print("\n\n")

In [ ]:
modelo = Anonimizacao.testar("Clinical-AI-Apollo/Medical-NER")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-DETAILED_DESCRIPTION', 'score': 0.23495772, 'index': 5, 'word': '▁cab', 'start': 9, 'end': 13}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.41113174, 'index': 6, 'word': 're', 'start': 13, 'end': 15}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.2657171, 'index': 7, 'word': 'ro', 'start': 15, 'end': 17}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.392915, 'index': 9, 'word': '▁a', 'start': 21, 'end': 23}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.51663315, 'index': 10, 'word': '▁moto', 'start': 23, 'end': 28}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.86315554, 'index': 11, 'word': 's', 'start': 28, 'end': 29}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.8784478, 'index': 12, 'word': 's', 'start': 29, 'end': 30}
{'entity': 'I-DETAILED_DESCRIPTION', 'score': 0.88795364, 'index': 13, 'word': 'era', 'start': 30,

In [ ]:
modelo = Anonimizacao.testar("dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-LOC', 'score': 0.64309186, 'index': 50, 'word': 'La', 'start': 114, 'end': 116}
{'entity': 'I-LOC', 'score': 0.9010507, 'index': 52, 'word': '##a', 'start': 118, 'end': 119}
{'entity': 'B-PER', 'score': 0.9656065, 'index': 56, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-PER', 'score': 0.98653424, 'index': 57, 'word': 'Co', 'start': 134, 'end': 136}

 Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da LOCgoLOC perto do PERta.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'I-ORG', 'score': 0.9160291, 'index': 3, 'word': 'Jo', 'start': 4, 'end': 6}
{'entity': 'I-ORG', 'score': 0.730859, 'index': 4, 'word': '##rg', 'start': 6, 'end': 8}
{'entity': 'I-ORG'

In [ ]:
modelo = Anonimizacao.testar("Davlan/bert-base-multilingual-cased-ner-hrl")

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-PER', 'score': 0.99537474, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'B-LOC', 'score': 0.97691196, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-LOC', 'score': 0.9885655, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-LOC', 'score': 0.97341466, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'I-LOC', 'score': 0.99005836, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 pessoa é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da local perto do local.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-PER', 'score': 0.9992424, 'index': 2, 'word': 'Jo', 'start': 4, 'end': 6}
{'entity': 'I-P

In [ ]:
modelo = Anonimizacao.testar("Babelscape/wikineural-multilingual-ner")

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-LOC', 'score': 0.96766573, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-LOC', 'score': 0.9977324, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-LOC', 'score': 0.99891067, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'I-LOC', 'score': 0.9993338, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da local perto do local.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-PER', 'score': 0.33752406, 'index': 2, 'word': 'Jo', 'start': 4, 'end': 6}
{'entity': 'I-PER', 'score': 0.2727368, 'index': 3, 'word': '##rgi', 'start': 6, 'end': 9}
{'entity': 'I-PER

In [ ]:
modelo = Anonimizacao.testar("lfcc/bert-portuguese-ner")

tokenizer_config.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-Pessoa', 'score': 0.9416293, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'B-Local', 'score': 0.83937794, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-Local', 'score': 0.60270226, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-Local', 'score': 0.60907286, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'I-Local', 'score': 0.64458346, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 B-Pessoa é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da B-Local perto do B-Local.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-Pessoa', 'score': 0.98107797, 'index': 2, 'word': 'Jo', 'start': 4, 'end

In [ ]:
modelo = Anonimizacao.testar("portugueseNLP/medialbertina_pt-pt_1.5b_NER")

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.64M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/74.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 


 Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 


 Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo





In [ ]:
modelo = Anonimizacao.testar("monilouise/ner_news_portuguese")

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at monilouise/ner_news_portuguese were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-PESSOA', 'score': 0.740456, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'B-LOC', 'score': 0.72580624, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-LOC', 'score': 0.89417595, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'L-LOC', 'score': 0.9265142, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'L-LOC', 'score': 0.9431214, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 B-PESSOA é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da local perto do local.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-PESSOA', 'score': 0.862516, 'index': 2, 'word': 'Jo', 'start': 4, 'end': 6}
{'entity

In [ ]:
modelo = Anonimizacao.testar("pedrogarcias/bert-large-portuguese-cased-finetuned-ner")

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'LABEL_1', 'score': 0.99997854, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'LABEL_1', 'score': 0.9991379, 'index': 2, 'word': 'é', 'start': 8, 'end': 9}
{'entity': 'LABEL_1', 'score': 0.9999789, 'index': 3, 'word': 'cab', 'start': 10, 'end': 13}
{'entity': 'LABEL_1', 'score': 0.99997747, 'index': 4, 'word': '##rero', 'start': 13, 'end': 17}
{'entity': 'LABEL_0', 'score': 0.7849881, 'index': 5, 'word': 'com', 'start': 18, 'end': 21}
{'entity': 'LABEL_0', 'score': 0.9949302, 'index': 6, 'word': 'a', 'start': 22, 'end': 23}
{'entity': 'LABEL_1', 'score': 0.9999583, 'index': 7, 'word': 'moto', 'start': 24, 'end': 28}
{'entity': 'LABEL_1', 'score': 0.99997544, 'index': 8, 'word': '##s', 'start': 28, 'end': 29}
{'entity': 'LABEL_1', 'score': 0.9999801, 'index': 9, 'word': '##ser', 'start': 29, 'end': 32}
{'entity': 'L

https://huggingface.co/liaad/NER_harem_bert-large-portuguese-cased

In [ ]:
modelo = Anonimizacao.testar("liaad/NER_harem_bert-large-portuguese-cased")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-PESSOA', 'score': 0.61345327, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'B-LOCAL', 'score': 0.8777183, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-LOCAL', 'score': 0.6885848, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-LOCAL', 'score': 0.95406175, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'I-LOCAL', 'score': 0.94550633, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 PESSOA é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da LOCAL perto do LOCAL.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-PESSOA', 'score': 0.8845814, 'index': 1, 'word': 'Seu', 'start': 0, 'end': 3

In [ ]:
modelo = Anonimizacao.testar("liaad/NER_harem_bert-base-portuguese-cased")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.





Joaquim é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da Lagoa perto do João Cota. 

{'entity': 'B-PESSOA', 'score': 0.8437839, 'index': 1, 'word': 'Joaquim', 'start': 0, 'end': 7}
{'entity': 'B-LOCAL', 'score': 0.9618173, 'index': 35, 'word': 'Lagoa', 'start': 114, 'end': 119}
{'entity': 'B-LOCAL', 'score': 0.9165109, 'index': 38, 'word': 'João', 'start': 129, 'end': 133}
{'entity': 'I-LOCAL', 'score': 0.9800228, 'index': 39, 'word': 'Co', 'start': 134, 'end': 136}
{'entity': 'I-LOCAL', 'score': 0.97896886, 'index': 40, 'word': '##ta', 'start': 136, 'end': 138}

 PESSOA é cabrero com a motossera, corto duas candeia cambará e tiro as arvores, sobro nadinha. Aqui no bairro da LOCAL perto do LOCAL.



Seu Jorgiscleisson tem CPF 345.234.231-2 e a placa do carro preto dele é DFR-034, ele botou fogo na floresta do bilico toda e tá fugindo 

{'entity': 'B-PESSOA', 'score': 0.8009951, 'index': 2, 'word': 'Jo', 'start': 4, 'end': 6}
{